In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import cv2

# Loading the dataset

In [4]:
#Define the classes and their labels
classes = ['AbdomenCT', 'ChestCT', 'HeadCT', 'ChestXRay', 'Hand', 'BreastMRI']

In [5]:
#Define the class directories
abdomen_DIR = '../FCV2/AbdomenCT'
breastmri_DIR = '../FCV2/BreastMRI'
chestct_DIR = '../FCV2/ChestCT'
cxr_DIR = '../FCV2/CXR'
hand_DIR = '../FCV2/Hand'
headct_DIR = '../FCV2/HeadCT'


In [6]:
# Creates lists of the image paths and their labels (we'll make a dataframe out of these)
X = []
y = []

def makeTrainData(className, classDIR):
    for img in os.listdir(classDIR):
        path = os.path.join(classDIR, img)
        X.append(path)
        y.append(className)

In [7]:
# Add each class to the X and y list

makeTrainData('AbdomenCT', abdomen_DIR)

makeTrainData('ChestCT', chestct_DIR)

makeTrainData('HeadCT', headct_DIR)

makeTrainData('ChestXRay', cxr_DIR)

makeTrainData('Hand', hand_DIR)

makeTrainData('BreastMRI', breastmri_DIR)

In [8]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_val_images, test_images, train_val_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the train_val data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_val_images, train_val_labels, test_size=0.2, random_state=42)

train_images += val_images
train_labels += val_labels


# Preprocessing and augmenting the data

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
data = pd.DataFrame({'path': train_images, 'class': train_labels})
testdata = pd.DataFrame({'path': test_images, 'class': test_labels})
data.head()

,path,class
0,../FCV2/CXR\000484.jpeg,ChestXRay
1,../FCV2/BreastMRI\000421.jpeg,BreastMRI
2,../FCV2/HeadCT\000755.jpeg,HeadCT
3,../FCV2/Hand\003880.jpeg,Hand
4,../FCV2/BreastMRI\002538.jpeg,BreastMRI


In [11]:
one_hot_encoded = pd.get_dummies(data['class'])
data = pd.concat([data, one_hot_encoded], axis=1)
data.head()

,path,class,AbdomenCT,BreastMRI,ChestCT,ChestXRay,Hand,HeadCT
0,../FCV2/CXR\000484.jpeg,ChestXRay,False,False,False,True,False,False
1,../FCV2/BreastMRI\000421.jpeg,BreastMRI,False,True,False,False,False,False
2,../FCV2/HeadCT\000755.jpeg,HeadCT,False,False,False,False,False,True
3,../FCV2/Hand\003880.jpeg,Hand,False,False,False,False,True,False
4,../FCV2/BreastMRI\002538.jpeg,BreastMRI,False,True,False,False,False,False


In [12]:
# Creating a datagen to augment the image with rotations, flips, rescales, shear and zoom
# A 80-20 split is being used for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Create the training generator
train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col='path',
    y_col='class',
    target_size=(227, 227),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Create the testing generator
validation_generator = datagen.flow_from_dataframe(
    dataframe=data,
    x_col="path",
    y_col="class",
    target_size=(227, 227),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=True
)


test_generator = datagen.flow_from_dataframe(
    dataframe=testdata,
    x_col="path",
    y_col="class",
    target_size=(227, 227),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

Found 3840 validated image filenames belonging to 6 classes.
Found 960 validated image filenames belonging to 6 classes.
Found 1200 validated image filenames belonging to 6 classes.


# Defining the model - Alexnet

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [14]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(256, kernel_size=(5,5), strides=(1,1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# Compile the model with Adam optimizer and Accuracy as our metric
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       1

# Training the Model

In [15]:
model.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
120/120 [==============================] - 207s 2s/step - loss: 0.8675 - accuracy: 0.6776 - val_loss: 0.2537 - val_accuracy: 0.9292
Epoch 2/5
120/120 [==============================] - 228s 2s/step - loss: 0.2163 - accuracy: 0.9331 - val_loss: 0.1271 - val_accuracy: 0.9667
Epoch 3/5
120/120 [==============================] - 209s 2s/step - loss: 0.1690 - accuracy: 0.9508 - val_loss: 0.1418 - val_accuracy: 0.9625
Epoch 4/5
120/120 [==============================] - 208s 2s/step - loss: 0.2700 - accuracy: 0.9279 - val_loss: 0.1091 - val_accuracy: 0.9698
Epoch 5/5
120/120 [==============================] - 211s 2s/step - loss: 0.1298 - accuracy: 0.9578 - val_loss: 0.0547 - val_accuracy: 0.9812


# Evaluating and testing the model

In [16]:
score = model.evaluate(test_generator, steps=40)
print(f"\nAccuracy: {round(score[1]*100, 2)}%")

40/40 [==============================] - 16s 406ms/step - loss: 0.0651 - accuracy: 0.9825

Accuracy: 98.25%


In [2]:
# Generate predictions for the test images
test_steps = test_generator.samples // 32
test_pred = model.predict(test_generator, steps=test_steps, verbose=2)

# Define a function to convert predicted labels to class names
def get_class_name(pred):
    idx = np.argmax(pred)
    for class_name, class_idx in test_generator.class_indices.items():
        if class_idx == idx:
            return class_name


# Define a function to plot an image and its predicted and ground truth labels
def plot_image(image, pred_label, true_label):
    plt.imshow(image)
    plt.title('Prediction: {}\nTrue label: {}'.format(pred_label, true_label))
    plt.axis('off')
    plt.show()

# Loop through the test images and plot them alongside their predicted and ground truth labels
for i in range(50):
    # Get the next test image and its true label
    image, true_label = test_generator.next()
    
    # Get the predicted label for this image
    pred_label = get_class_name(test_pred[i])
    
    # Plot the image and its predicted and true labels
    plot_image(image[0], pred_label, true_label[0])


NameError: name 'test_generator' is not defined